In [1]:
from DB.Chromadb import ChromaDB
from langchain.schema import Document
from utils import get_only_paragraphs, get_all_datas, create_documents
BASE_DIR = 'datas' # 데이터가 저장돼 있는 루트를 의미합니다.

df = get_only_paragraphs(BASE_DIR)
# df = get_all_datas(BASE_DIR) # 이미지와 테이블이 포함된 전체 데이터도 가져올 수 있습니다.
df[:3]


,id,type,dir_route,original_content,file_name,page,investment,company_name,summary,image_route,table
0,5,paragraph,datas/카카오뱅크_한국신용평가(2024.09.13),김경근 선임애널리스트 02-787-2383 kyungkeun.kim@kisratin...,카카오뱅크_한국신용평가(2024.09.13).pdf,1,한국신용평가(2024.09.13),카카오뱅크,김경근 선임애널리스트가 쓴 카카오뱅크 관련 한국신용평가 리포트 입니다.,NaN,NaN
1,6,paragraph,datas/카카오뱅크_한국신용평가(2024.09.13),위 지원 실장 02-787-2279 geewon.wee@kisrating.com,카카오뱅크_한국신용평가(2024.09.13).pdf,1,한국신용평가(2024.09.13),카카오뱅크,"위 지원 실장, ***-***-****, g*********@*********.**...",NaN,NaN
6,16,paragraph,datas/카카오뱅크_한국신용평가(2024.09.13),"주) 총자산, 대출채권, 예수부채, 자기자본, 당기순이익은 은행계정 기준임 자료: ...",카카오뱅크_한국신용평가(2024.09.13).pdf,1,한국신용평가(2024.09.13),카카오뱅크,"카카오뱅크의 자산, 대출채권, 예수부채, 자기자본, 당기순이익은 은행계정 기준이며,...",NaN,NaN


In [2]:
# 데이터프레임의 행들을 각각 Document타입으로 바꾼 뒤, 리스트에 담습니다.
documents = create_documents(df)
documents[:3]

[Document(metadata={'id': 5, 'type': 'paragraph', 'dir_route': 'datas/카카오뱅크_한국신용평가(2024.09.13)', 'original_content': '김경근 선임애널리스트 02-787-2383 kyungkeun.kim@kisrating.com', 'file_name': '카카오뱅크_한국신용평가(2024.09.13).pdf', 'page': 1, 'investment': '한국신용평가(2024.09.13)', 'company_name': '카카오뱅크', 'image_route': nan, 'table': nan}, page_content='김경근 선임애널리스트가 쓴 카카오뱅크 관련 한국신용평가 리포트 입니다.'),
 Document(metadata={'id': 6, 'type': 'paragraph', 'dir_route': 'datas/카카오뱅크_한국신용평가(2024.09.13)', 'original_content': '위 지원 실장 02-787-2279 geewon.wee@kisrating.com', 'file_name': '카카오뱅크_한국신용평가(2024.09.13).pdf', 'page': 1, 'investment': '한국신용평가(2024.09.13)', 'company_name': '카카오뱅크', 'image_route': nan, 'table': nan}, page_content='위 지원 실장, ***-***-****, g*********@*********.***으로 연락 가능한 한국신용평가에서 카카오뱅크에 대한 리포트를 발표함.'),
 Document(metadata={'id': 16, 'type': 'paragraph', 'dir_route': 'datas/카카오뱅크_한국신용펴

In [3]:
# add_docs를 통해 문서들을 저장합니다.

collection_name = 'chrdb.db'
DB = ChromaDB(collection_name)
DB.add_docs(documents)



In [4]:
# verify_db를 통해 db내의 데이터 수를 확인할 수 있고, db객체를 받을 수 있습니다.
db = DB.verify_db()


Total documents in collection 'chrdb.db': 388


In [5]:
# 위처럼 import하여 kiwi tokenizer을 사용하는 bm25와 dpr, 앙상블 리트리버를 생성할 수 있습니다.
from retrievals import bm25, dpr, ensemble

topk = 3

DPRRetriever = dpr(db, topk = 3)
BM25Retriever = bm25(documents, topk = 3)

retrievals = [DPRRetriever, BM25Retriever]
weights = [0.5, 0.5]
search_type = 'mmr'
ensemble_retriever = ensemble(retrievals, topk = topk, weights = weights, search_type = search_type)



In [6]:
# 다음과 같이 retriever.invoke(query)를 통해 관련 문서를 검색할 수 있습니다.
query = '네이버의 2024년 매출을 알려줘.'

DPRRetriever.invoke(query)

[Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 19, 'investment': '하나증권(2024.10.10)', 'original_content': '네이버는 2024년 연결 기준 영업수익 10조 5,184억원(+8.8%YoY), 영업이익 1조 8,994억원 (+27.6%YoY, OPM 18.1%)을 기록할 것으로 전망한다. 서치플랫폼 매출액은 3조 8,546억원 (+7.4%YoY)으로 추정한다. DA 광고 지면 증가 효과는 4분기까지 높은 성장률로 확인될 전 망이다. 커머스 매출액은 2조 9,045억원(+14.1%YoY)으로 전망한다. 네이버는 연내 모바일 쇼핑 탭을 네이버플러스 스토어로 개편할 계획이다. 스마트/브랜드스토어를 중심으로한 개 인화 추천, UI/UX 개선이 도입될 예정이다. 성장 둔화를 겪고 있는 커머스 광고 매출과 커 머스 GMV의 반등을 기대한다.', 'page': 1, 'type': 'paragraph'}, page_content='네이버는 2024년 연결 기준 영업수익 10조 5,184억원, 영업이익 1조 8,994억원을 기록할 것으로 전망하며, 특히 쇼핑 탭을 네이버플러스 스토어로 개편하여 개인화 추천 및 UI/UX 개선을 통해 커머스 광고 매출과 GMV의 반등을 기대함.'),
 Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 45, 'investment': '하나증권(2024.10.10)', 'original_content': '주: 2Q21 왓패드 인수, 4Q21

In [7]:
BM25Retriever.invoke(query)

[Document(metadata={'id': 19, 'type': 'paragraph', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'original_content': '네이버는 2024년 연결 기준 영업수익 10조 5,184억원(+8.8%YoY), 영업이익 1조 8,994억원 (+27.6%YoY, OPM 18.1%)을 기록할 것으로 전망한다. 서치플랫폼 매출액은 3조 8,546억원 (+7.4%YoY)으로 추정한다. DA 광고 지면 증가 효과는 4분기까지 높은 성장률로 확인될 전 망이다. 커머스 매출액은 2조 9,045억원(+14.1%YoY)으로 전망한다. 네이버는 연내 모바일 쇼핑 탭을 네이버플러스 스토어로 개편할 계획이다. 스마트/브랜드스토어를 중심으로한 개 인화 추천, UI/UX 개선이 도입될 예정이다. 성장 둔화를 겪고 있는 커머스 광고 매출과 커 머스 GMV의 반등을 기대한다.', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'page': 1, 'investment': '하나증권(2024.10.10)', 'company_name': '네이버', 'image_route': nan, 'table': nan}, page_content='네이버는 2024년 연결 기준 영업수익 10조 5,184억원, 영업이익 1조 8,994억원을 기록할 것으로 전망하며, 특히 쇼핑 탭을 네이버플러스 스토어로 개편하여 개인화 추천 및 UI/UX 개선을 통해 커머스 광고 매출과 GMV의 반등을 기대함.'),
 Document(metadata={'id': 11, 'type': 'paragraph', 'dir_route': 'datas/롯데렌탈_하나증권(2024.09.20)', 'original_content': '도 표 1. 2024년 9월 19일 기준 쏘카 주주 구성 및 지분율', 'file_name': '롯데렌탈_하나증권

In [15]:
# 어째서인지 Ensemble retrieval은 topk가 먹질 않습니다.. ㅋㅋ
ensemble_retriever.invoke(query)

[Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 19, 'investment': '하나증권(2024.10.10)', 'original_content': '네이버는 2024년 연결 기준 영업수익 10조 5,184억원(+8.8%YoY), 영업이익 1조 8,994억원 (+27.6%YoY, OPM 18.1%)을 기록할 것으로 전망한다. 서치플랫폼 매출액은 3조 8,546억원 (+7.4%YoY)으로 추정한다. DA 광고 지면 증가 효과는 4분기까지 높은 성장률로 확인될 전 망이다. 커머스 매출액은 2조 9,045억원(+14.1%YoY)으로 전망한다. 네이버는 연내 모바일 쇼핑 탭을 네이버플러스 스토어로 개편할 계획이다. 스마트/브랜드스토어를 중심으로한 개 인화 추천, UI/UX 개선이 도입될 예정이다. 성장 둔화를 겪고 있는 커머스 광고 매출과 커 머스 GMV의 반등을 기대한다.', 'page': 1, 'type': 'paragraph'}, page_content='네이버는 2024년 연결 기준 영업수익 10조 5,184억원, 영업이익 1조 8,994억원을 기록할 것으로 전망하며, 특히 쇼핑 탭을 네이버플러스 스토어로 개편하여 개인화 추천 및 UI/UX 개선을 통해 커머스 광고 매출과 GMV의 반등을 기대함.'),
 Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 45, 'investment': '하나증권(2024.10.10)', 'original_content': '주: 2Q21 왓패드 인수, 4Q21

In [12]:
ensemble_retriever.invoke(query, search_kwargs={'k':3})

[Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 19, 'investment': '하나증권(2024.10.10)', 'original_content': '네이버는 2024년 연결 기준 영업수익 10조 5,184억원(+8.8%YoY), 영업이익 1조 8,994억원 (+27.6%YoY, OPM 18.1%)을 기록할 것으로 전망한다. 서치플랫폼 매출액은 3조 8,546억원 (+7.4%YoY)으로 추정한다. DA 광고 지면 증가 효과는 4분기까지 높은 성장률로 확인될 전 망이다. 커머스 매출액은 2조 9,045억원(+14.1%YoY)으로 전망한다. 네이버는 연내 모바일 쇼핑 탭을 네이버플러스 스토어로 개편할 계획이다. 스마트/브랜드스토어를 중심으로한 개 인화 추천, UI/UX 개선이 도입될 예정이다. 성장 둔화를 겪고 있는 커머스 광고 매출과 커 머스 GMV의 반등을 기대한다.', 'page': 1, 'type': 'paragraph'}, page_content='네이버는 2024년 연결 기준 영업수익 10조 5,184억원, 영업이익 1조 8,994억원을 기록할 것으로 전망하며, 특히 쇼핑 탭을 네이버플러스 스토어로 개편하여 개인화 추천 및 UI/UX 개선을 통해 커머스 광고 매출과 GMV의 반등을 기대함.'),
 Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 45, 'investment': '하나증권(2024.10.10)', 'original_content': '주: 2Q21 왓패드 인수, 4Q21

In [ ]:
from evaluation import retrieval_evaluate
import pandas as pd
eval_dataset = pd.read_csv('datas/validation_dataset.csv')

# retrieval과 evaldataset을 통해 acc를 평가해볼 수 있습니다.
retrieval_evaluate(retrieval=BM25Retriever, eval_dataset = eval_dataset)

acc: 0.8165
